<a href="https://colab.research.google.com/github/laribar/bitcoinprediction/blob/main/modelo_yfinance_NASDAQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error
import logging
from scipy.stats import norm

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Função para baixar dados e calcular métricas técnicas
def baixar_dados(acao, intervalo="1d", periodo="2y"):
    try:
        df = yf.download(acao, period=periodo, interval=intervalo, progress=False, auto_adjust=False)
        if df.empty or df.isnull().values.any():
            logging.error(f"Erro: Dados faltantes ou inválidos para {acao}.")
            return None

        preco_coluna = "Adj Close" if "Adj Close" in df.columns else "Close"

        if preco_coluna not in df.columns:
            logging.error(f"Erro ao processar {acao}: Nenhuma coluna de fechamento encontrada nos dados.")
            return None

        # Remove NaNs
        df.dropna(inplace=True)

        # Calcula a média móvel simples (SMA) e desvio padrão com shift para evitar forward-looking bias
        df["SMA"] = df[preco_coluna].rolling(window=20).mean().shift(1)
        df["STD"] = df[preco_coluna].rolling(window=20).std().shift(1)

        # Calcula as bandas superior e inferior
        df["Upper_Band"] = df["SMA"] + (df["STD"] * 2)
        df["Lower_Band"] = df["SMA"] - (df["STD"] * 2)

        # Calcula suporte e resistência com shift
        df["Support"] = df["Low"].rolling(window=20).min().shift(1)
        df["Resistance"] = df["High"].rolling(window=20).max().shift(1)

        # Calcula LTAs e LTBs com lookback dinâmico baseado na volatilidade
        lookback = max(10, int(df["STD"].mean() * 5))  # Ajuste baseado na volatilidade
        df["LTA"] = df["High"].rolling(window=lookback).max().shift(1)
        df["LTB"] = df["Low"].rolling(window=lookback).min().shift(1)

        # Remove colunas auxiliares que não serão usadas no modelo
        df = df.drop(columns=["STD"])

        return df

    except Exception as e:
        logging.error(f"Erro ao baixar dados para {acao}: {e}", exc_info=True)
        return None

# Função para calcular a probabilidade de atingir o preço alvo
def calcular_probabilidade(preco_atual, preco_alvo, volatilidade, dias):
    """
    Calcula a probabilidade de atingir o preço alvo com base na volatilidade e no horizonte de tempo.
    """
    # Garantir que preco_atual e volatilidade sejam valores escalares
    if isinstance(preco_atual, pd.Series):
        preco_atual = preco_atual.item()  # Converte para valor escalar
    if isinstance(volatilidade, pd.Series):
        volatilidade = volatilidade.item()  # Converte para valor escalar

    if preco_atual == 0 or volatilidade == 0:
        return 0.0

    T = dias / 252  # Converter dias para anos (252 dias úteis no ano)
    d1 = (np.log(preco_alvo / preco_atual)) / (volatilidade * np.sqrt(T))
    probabilidade = norm.cdf(d1)
    return probabilidade

# Função para treinar o modelo e prever preços futuros com probabilidades
def prever_precos_com_probabilidade(df, acao, intervalo):
    try:
        # Definir target como o preço do dia seguinte
        df["Target"] = df["Close"].shift(-1)
        df.dropna(inplace=True)  # Remove a última linha com target NaN

        # Selecionar features e target
        X = df[['SMA', 'Upper_Band', 'Lower_Band', 'Support', 'Resistance', 'LTA', 'LTB']]
        y = df['Target'].values.ravel()  # Converter y para array 1D

        # Separar os últimos 10% dos dados para teste
        test_size = int(0.1 * len(df))
        X_train, X_test = X.iloc[:-test_size], X.iloc[-test_size:]
        y_train, y_test = y[:-test_size], y[-test_size:]

        # Verificar se há dados suficientes para treinamento e teste
        if len(X_train) == 0 or len(X_test) == 0:
            logging.error(f"Erro: Dados insuficientes para treinamento/teste em {acao} no intervalo {intervalo}")
            return None, None

        # Padronizar os dados
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Ajuste de hiperparâmetros com GridSearchCV e TimeSeriesSplit
        param_grid = {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5, 10]
        }
        model = RandomForestRegressor(random_state=42)
        tss = TimeSeriesSplit(n_splits=5)  # Usar 5 splits para validação cruzada
        grid_search = GridSearchCV(model, param_grid, cv=tss, scoring='neg_mean_squared_error')
        grid_search.fit(X_train_scaled, y_train)

        best_model = grid_search.best_estimator_

        # Avaliar o modelo no conjunto de teste
        y_pred = best_model.predict(X_test_scaled)
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        logging.info(f"Desempenho do modelo para {acao}: MSE = {mse:.2f}, MAE = {mae:.2f}")

        # Criar datas futuras
        future_dates = [df.index[-1] + pd.Timedelta(days=i) for i in range(1, 6)]

        # Criar features futuras com base em tendências
        future_features = pd.DataFrame({
            'SMA': [df['SMA'].iloc[-1] + i * 0.1 for i in range(5)],
            'Upper_Band': [df['Upper_Band'].iloc[-1] + i * 0.1 for i in range(5)],
            'Lower_Band': [df['Lower_Band'].iloc[-1] + i * 0.1 for i in range(5)],
            'Support': [df['Support'].iloc[-1] + i * 0.1 for i in range(5)],
            'Resistance': [df['Resistance'].iloc[-1] + i * 0.1 for i in range(5)],
            'LTA': [df['LTA'].iloc[-1] + i * 0.1 for i in range(5)],
            'LTB': [df['LTB'].iloc[-1] + i * 0.1 for i in range(5)]
        }, index=future_dates)

        # Transformar features futuras usando o scaler ajustado
        future_features_scaled = scaler.transform(future_features)

        # Fazer previsões
        future_predictions = best_model.predict(future_features_scaled)

        # Calcular volatilidade histórica (desvio padrão dos retornos diários)
        retornos = df['Close'].pct_change().dropna()
        volatilidade = retornos.std() * np.sqrt(252)  # Volatilidade anualizada

        # Calcular probabilidades para cada previsão
        probabilidades = []
        for i, previsao in enumerate(future_predictions):
            dias = i + 1  # Horizonte de tempo em dias
            probabilidade = calcular_probabilidade(df['Close'].iloc[-1], previsao, volatilidade, dias)
            probabilidades.append((previsao, probabilidade, dias))

        return future_predictions, probabilidades

    except Exception as e:
        logging.error(f"Erro ao prever preços para {acao}: {e}", exc_info=True)
        return None, None

# Função para gerar alertas de compra/venda com probabilidades
def gerar_alertas(df, previsoes, probabilidades, acao, intervalo):
    try:
        # Garantir que last_price seja um valor escalar
        last_price = df['Close'].iloc[-1]
        if isinstance(last_price, pd.Series):
            last_price = last_price.item()  # Converte para um valor escalar

        if previsoes is not None and len(previsoes) > 0:
            avg_future_price = np.mean(previsoes)

            # Garantir que todas as métricas sejam valores escalares
            sma = df['SMA'].iloc[-1].item() if isinstance(df['SMA'].iloc[-1], pd.Series) else df['SMA'].iloc[-1]
            upper_band = df['Upper_Band'].iloc[-1].item() if isinstance(df['Upper_Band'].iloc[-1], pd.Series) else df['Upper_Band'].iloc[-1]
            lower_band = df['Lower_Band'].iloc[-1].item() if isinstance(df['Lower_Band'].iloc[-1], pd.Series) else df['Lower_Band'].iloc[-1]
            support = df['Support'].iloc[-1].item() if isinstance(df['Support'].iloc[-1], pd.Series) else df['Support'].iloc[-1]
            resistance = df['Resistance'].iloc[-1].item() if isinstance(df['Resistance'].iloc[-1], pd.Series) else df['Resistance'].iloc[-1]
            lta = df['LTA'].iloc[-1].item() if isinstance(df['LTA'].iloc[-1], pd.Series) else df['LTA'].iloc[-1]
            ltb = df['LTB'].iloc[-1].item() if isinstance(df['LTB'].iloc[-1], pd.Series) else df['LTB'].iloc[-1]

            # Definir ordens de compra e venda
            ordem_compra = max(support, lower_band)  # Ordem de compra: máximo entre suporte e banda inferior
            ordem_venda = min(resistance, upper_band)  # Ordem de venda: mínimo entre resistência e banda superior

            # Gerar alerta e justificativa
            if avg_future_price > last_price * 1.02:  # Tendência de alta
                alerta = '🚀 Compra sugerida'
                justificativa = (
                    f"A média das previsões futuras ({avg_future_price:.2f}) é 2% maior que o preço atual ({last_price:.2f}). "
                    f"Além disso, a SMA está em {sma:.2f}, a banda superior em {upper_band:.2f}, e a banda inferior em {lower_band:.2f}. "
                    f"O suporte está em {support:.2f}, a resistência em {resistance:.2f}, e a LTA em {lta:.2f}. "
                    f"Isso indica uma tendência de alta."
                )
                # Exibir valores de compra e venda somente se a sugestão for "compra"
                print(f"Ordem de Compra: {ordem_compra:.2f}")
                print(f"Ordem de Venda: {ordem_venda:.2f}")
            elif avg_future_price < last_price * 0.98:  # Tendência de baixa
                alerta = '⚠️ Venda sugerida'
                justificativa = (
                    f"A média das previsões futuras ({avg_future_price:.2f}) é 2% menor que o preço atual ({last_price:.2f}). "
                    f"Além disso, a SMA está em {sma:.2f}, a banda superior em {upper_band:.2f}, e a banda inferior em {lower_band:.2f}. "
                    f"O suporte está em {support:.2f}, a resistência em {resistance:.2f}, e a LTB em {ltb:.2f}. "
                    f"Isso indica uma tendência de baixa."
                )
            else:
                alerta = '🔍 Acompanhar'
                justificativa = (
                    f"A média das previsões futuras ({avg_future_price:.2f}) está dentro de 2% do preço atual ({last_price:.2f}). "
                    f"A SMA está em {sma:.2f}, a banda superior em {upper_band:.2f}, e a banda inferior em {lower_band:.2f}. "
                    f"O suporte está em {support:.2f}, a resistência em {resistance:.2f}, a LTA em {lta:.2f}, e a LTB em {ltb:.2f}. "
                    f"Isso sugere que o mercado está estável no momento."
                )
        else:
            alerta = '❌ Sem previsões disponíveis'
            justificativa = 'Não há previsões disponíveis para esta ação.'
            ordem_compra = None
            ordem_venda = None

        # Exibir resultados
        print(f"Stock: {acao} | Intervalo: {intervalo}")
        print(f"Last price: {last_price:.2f}")
        print(f"Predictions: {previsoes}")
        print(f"Alert: {alerta}")
        print(f"Justificativa: {justificativa}")

        # Exibir probabilidades
        if probabilidades is not None:
            print("\nProbabilidades de Atingir o Preço Alvo:")
            for previsao, probabilidade, dias in probabilidades:
                print(f"Previsão: {previsao:.2f} | Probabilidade: {probabilidade * 100:.2f}% | Dias: {dias}")

        print("-" * 80)

    except Exception as e:
        logging.error(f"Erro ao gerar alertas para {acao}: {e}", exc_info=True)

# Função para obter o Top 10 da NASDAQ por capitalização de mercado
def obter_top_10_nasdaq():
    """
    Obtém o Top 10 da NASDAQ por capitalização de mercado.
    """
    # Baixar a lista de tickers da NASDAQ
    nasdaq_tickers = pd.read_html("https://en.wikipedia.org/wiki/Nasdaq-100")[4]
    top_10 = nasdaq_tickers.head(10)['Ticker'].tolist()
    return top_10

# Processar ações
acoes = obter_top_10_nasdaq()
for acao in acoes:
    df = baixar_dados(acao)
    if df is not None:
        previsoes, probabilidades = prever_precos_com_probabilidade(df, acao, "1d")
        gerar_alertas(df, previsoes, probabilidades, acao, "1d")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Ordem de Compra: 381.27
Ordem de Venda: 461.00
Stock: ADBE | Intervalo: 1d
Last price: 389.61
Predictions: [460.79619843 460.79619843 460.79619843 460.79619843 460.79619843]
Alert: 🚀 Compra sugerida
Justificativa: A média das previsões futuras (460.80) é 2% maior que o preço atual (389.61). Além disso, a SMA está em 429.74, a banda superior em 478.21, e a banda inferior em 381.27. O suporte está em 374.50, a resistência em 461.00, e a LTA em 557.90. Isso indica uma tendência de alta.

Probabilidades de Atingir o Preço Alvo:
Previsão: 460.80 | Probabilidade: 100.00% | Dias: 1
Previsão: 460.80 | Probabilidade: 100.00% | Dias: 2
Previsão: 460.80 | Probabilidade: 100.00% | Dias: 3
Previsão: 460.80 | Probabilidade: 99.99% | Dias: 4
Previsão: 460.80 | Probabilidade: 99.97% | Dias: 5
--------------------------------------------------------------------------------


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Stock: AMD | Intervalo: 1d
Last price: 107.14
Predictions: [106.94969257 108.36567486 108.09164907 106.8872902  106.90975676]
Alert: 🔍 Acompanhar
Justificativa: A média das previsões futuras (107.44) está dentro de 2% do preço atual (107.14). A SMA está em 102.43, a banda superior em 111.74, e a banda inferior em 93.12. O suporte está em 94.73, a resistência em 116.55, a LTA em 119.85, e a LTB em 94.73. Isso sugere que o mercado está estável no momento.

Probabilidades de Atingir o Preço Alvo:
Previsão: 106.95 | Probabilidade: 47.56% | Dias: 1
Previsão: 108.37 | Probabilidade: 60.89% | Dias: 2
Previsão: 108.09 | Probabilidade: 56.97% | Dias: 3
Previsão: 106.89 | Probabilidade: 48.38% | Dias: 4
Previsão: 106.91 | Probabilidade: 48.68% | Dias: 5
--------------------------------------------------------------------------------
